# Intro Unsupervised Learning

![](https://media.giphy.com/media/kZJcpM3gYdlK4D64Ft/giphy.gif)

## When do we use Unsupervised Learning?

**Examples:** (When there are no labels)

+ Customer segmentation
+ Product segmentation
+ One-person segment (what works for one group doesn't necessarily work for another)
+ Grouping according to characteristics

**Situation**

Each element (user, customer, product...) can be considered a vector. Given a package of vectors, they are distributed by clusters. Similar vectors will be in the same cluster, and similar clusters will be 'closer' to each other than those that are more different.

All this proceeding assumes the existence of a distance metric, so that distance between vectors within the vector space can be measured.

**Steps to apply USL:**

+ **1**: Vector representation
+ **2**: Metric, distance function
+ **3**: Similarity measure

When creating a cluster, the vector in the center of the cluster (centroid) is considered the prototype or the most representative of the cluster.


**Problems treatable by USL:**

+ Market segmentation
+ Detection of fraud in insurance or banking
+ Response to medical treatments
+ Outlier detection

**Examples of models:**

+ K-Means
+ Hierarchical Clustering
+ DBSCAN
+ HDBSCAN
+ GMM

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


#Viz mantra

%matplotlib inline
%config Inlinebackend.figure_format= 'retina'
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12.,6.)})
sns.set_style("whitegrid")

## PCA
**Main Component Analysis**
Principal Component Analysis (PCA) is a statistical method that allows simplifying the complexity of sample spaces with many dimensions while preserving their information. Suppose there is a sample with n individuals each with p variables (X1, X2, …, Xp), that is, the sample space has p dimensions. PCA allows to find a number of underlying factors (z<p) that explain approximately the same as the original p variables. Where before p values ​​were needed to characterize each individual, now z values ​​suffice. Each of these z new variables is called the principal component.

Principal Component Analysis belongs to the family of techniques known as unsupervised learning. The supervised learning methods described in previous chapters have the objective of predicting a response variable Y from a series of predictors. To do this, we have p characteristics (X1, X2 … Xp) and the response variable Y measured in n observations. In the case of unsupervised learning, the response variable Y is not taken into account since the objective is not to predict Y but to extract information using the predictors, for example, to identify subgroups. The main problem faced by unsupervised learning methods is the difficulty in validating the results given that there is no response variable available that allows them to be contrasted.

The PCA method therefore allows "condensing" the information provided by multiple variables into just a few components. This makes it a very useful method to apply after using other statistical techniques such as regression, clustering... Even so, we must not forget that it is still necessary to have the value of the original variables to calculate the components.

In [ ]:
iris = sns.load_dataset("iris")

In [ ]:
iris.head()

In [ ]:
iris.species.unique()

In [ ]:
# PCA using Sklearn
sns.pairplot(iris, hue="species")

In [ ]:
from sklearn.decomposition import PCA

The result after doing fit_transform is that where we had 4 features we now have 2

In [ ]:
iris.head()

In [ ]:
pca = PCA(2)
iris_t = pd.DataFrame(pca.fit_transform(iris.drop(columns="species")), columns=["PC1", "PC2"])

In [ ]:
iris_t.head()

In [ ]:
# DF: PC1, PC2 & Species
complete = iris_t.copy()
complete["species"] = iris["species"]

In [ ]:
complete.head()

## Deciding on the amount of components

In [ ]:
# Initializing
pca = PCA(n_components=None)
pca.fit(iris.drop(columns="species"))

# Calculating variance
exp_var = pca.explained_variance_ratio_ * 100
cum_exp_var = np.cumsum(exp_var)

# Plotting
plt.bar(range(1, 5), exp_var, align='center',label='Individual explained variance') #bars
plt.step(range(1, 5), cum_exp_var, where='mid', label='Cumulative explained variance', color='red') #line

# Axis
plt.ylabel('Explained variance percentage')
plt.xlabel('Principal component index')
plt.xticks(ticks=[1, 2, 3, 4])
plt.legend(loc='best')
plt.tight_layout()

pca.components has shape `[n_components, n_features]`

In [ ]:
pca.components_ 

## K-Means


The oldest and the most popular. The idea is to previously select how many clusters are desired ($k$). The central points of the cluster (centroids) are selected randomly. For each new record, a cluster is reassigned and the centroid (mean) is recalculated. The process is iterated until there is no change in clustering.

![kmeans](../images/kmeans.png)

![kmeans2](../images/kmeans2.png)

Wait didn't we see KNN already?

KNN is unsupervised for classification. K-means is unsupervised for clustering

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd

In [ ]:
pca_iris = sns.scatterplot(iris_t["PC1"],iris_t["PC2"], hue=iris["species"]);

## We standardize the data

**Since KMeans is based on distance** and our variables have very different variances, even though they have the same unit, it would be a good idea to standardize the data if we had not done PCA.
Anyway, I leave you an example of how the data would look.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
dat_ = iris.copy()

In [ ]:
dat_.drop(columns="species", inplace=True)

In [ ]:
dat_.head()

In [ ]:
scaler = StandardScaler()
df_scaler = pd.DataFrame(scaler.fit_transform(dat_), columns=dat_.columns)
df_scaler.head()

## Train the algorithm

### K-Means with the two PCA

Since we are testing and understanding the algorithm, we are going to enter the number of clusters by hand

In [ ]:
km = KMeans(n_clusters=3)
km.fit(iris_t)

In [ ]:
y_pred = km.predict(iris_t)

In [ ]:
iris_t["predict"] = y_pred

In [ ]:
iris_t["real"] = iris["species"]

In [ ]:
iris_t.sample(10)

In [ ]:
crosstab = pd.crosstab(iris_t.predict, iris_t.real)

In [ ]:
crosstab

### K-Means with all the features

In [ ]:
km2  = KMeans(n_clusters=3)
km2.fit(df_scaler)

In [ ]:
y_four = km2.predict(df_scaler)

In [ ]:
df_scaler["real"] = iris.species
df_scaler["kmeans"] = y_four

In [ ]:
df_scaler.head()

In [ ]:
cross2 = pd.crosstab(df_scaler.kmeans, df_scaler.real)
cross2

## Let's plot it

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,7))
pca_iris = sns.scatterplot(iris_t["PC1"],iris_t["PC2"], hue=iris_t["predict"], ax=axs[0]);
pca_iris = sns.scatterplot(iris_t["PC1"],iris_t["PC2"], hue=iris_t["real"], ax=axs[1]);

## We measure the model / (Silhouette Score)
silhouette punctuation
The silhouette score is a metric of the separation of the clusters.

It ranges from -1 to 1, where negative values ​​mean that the clusters are misallocated, 0 means that the clusters overlap, and 1 means that the clusters are separated and defined.

In [ ]:
from sklearn.metrics import silhouette_score

### Score of the kmeans with the data WITH PCA

In [ ]:
iris_t_onlypcs = iris_t.drop(columns=["predict", "real"])

In [ ]:
silhouette_score(iris_t_onlypcs, y_pred)

### Score of the kmeans with the data WITHOUT PCA

In [ ]:
df_scaler_features = df_scaler.drop(columns=["real", "kmeans"])

In [ ]:
silhouette_score(df_scaler_features, y_four)

## Elbow method
Probably the best known method, where you calculate and plot the sum of squares at each number of clusters, and there you look for a change in slope from steep to shallow to determine the optimal number of clusters. This method is inaccurate, but still potentially useful.

The elbow curve method is useful because it shows how increasing the number of clusters contributes to separating the clusters in a significant, not marginal, way. The curve indicates that additional groups beyond the third have little value. The elbow method is pretty neat, but a naive solution based on intracluster variance. The gap statistic is a more sophisticated method of dealing with data that has a distribution with no obvious clustering.

This method works as follows: the sum of squared errors within the cluster is calculated for different values ​​of K and the K is chosen for which the sum of squared errors begins to decrease. This is visible as an elbow.

Sum within a group of squared errors sounds a bit complex. Let's break it down:

The squared error for each point is the square of the point's distance from its plot, that is, its predicted cluster center.
The sum of squared errors score is the sum of these squared errors for all points.
Any distance metric can be used, such as the Euclidean distance or the Manhattan distance.
Now, to apply the elbow method and obtain the optimal number of clusters, it can be done as follows:

Compute the clustering algorithm for different values ​​of K. For example, varying K from 1 to 10 clusters.
For each K, compute the total sum of squares within the cluster.
Plot the curve of the sum of squared errors according to the number of groups K.
The location of a curve, elbow, on the graph is generally considered to be an indicator of the appropriate number of groups.

**Inertia error method**: The more variety there is between the observations in the dataset, the greater their distances from their associated centroids. The inertia or intertia, in the context of K-Means, is the sum of all the distances of the observations of a cluster to its centroid. Assuming that the objective is to reduce the sum of distances (squares) of points with their respective centroids, the lower this total sum, the better, since it will indicate a greater homogeneity in the observations belonging to the created clusters.

In [ ]:
kmeans = [KMeans(n_clusters=i) for i in range(1,21)]
for model in kmeans:
    model.fit(dat_)

In [ ]:
inertias = [model.inertia_ for model in kmeans]
inertias[:3]

In [ ]:
plt.plot(range(1,21),inertias)
plt.xticks(range(1,21));

We see that the exact cluster point we need to do for our data is 3